
# Projet: Facebook BaBi tasks
Notebook de la semaine du 22/03
Par Thierry Loesch et Bryce Tichit

Comme demandé par notre tuteur, nous allons désormais essayer d'adapter les travaux fait jusqu'ici sur un autre projet sensiblement identique.

-----------------------------------

5.1  Notation automatique de réponses courtes à des questions

Les données sont ici pour le téléchargement et leur description est dans cet article

Lorsque l'on connait:

    une question,
    une réponse d'un étudiant,
    la bonne réponse, 

comment prédire la note à donné à la réponse de l'étudiant. Il peut être aussi intéressant d'essayer d'apprendre à générer la réponse. 

--------------------------

On cherchera en premier à prédire la note de l'étudiant, le réseau devra rendre un réel. Normaliser les notes (les mettre entre 0 et 1) et activation par Sigmoid (output entre 0 et 1) afin de prédire la note.


TODO:

- Parser données
- Vectorisation
- Modèle

In [15]:
import re
from keras.utils.data_utils import get_file
import zipfile
import string

def tokenize(sent):
    
    def remPunctuation(sent):
        return "".join(l for l in sent if l not in string.punctuation)
        
    return [x.strip() for x in re.split('(\W+)?', remPunctuation(sent.lower())) if x.strip()]

def parseData(questions,answers,student_answers=''):
    
    #questions_data = dict() #ce dictionnaire contiendra les tuples (question,answer) indexés par la clé "numéro de question"
    
    questions_data = dict()
    
    for line in questions.split('\n'): #une ligne = une question
        line = line.decode('utf-8').strip()
        try:
            index,question = line.split(' ',1)
        except:
            continue
        
        questions_data[index] = [question.replace('<STOP>',''),None]
    
    for line in answers.split('\n'):
        line = line.decode('utf-8').strip()
        
        try:
            index,answer = line.split(' ',1)
        except:
            continue
            
        questions_data[index][1] = answer.replace('<STOP>','')
        
    questions_data = {k: tuple(map(tokenize,v)) for k, v in questions_data.items()}
    
    stud_ans = dict()
    
    for line in student_answers.split('\n'):
        line = line.decode('utf-8')
        try:
            index,ans = line.split(' ',1)
        except:
            continue
            
        answers=ans.split('<STOP>')
        answers=[a for a in answers if a]
            
        if index in stud_ans:
            stud_ans[index] += map(tokenize,answers)
        else:
            stud_ans[index] = map(tokenize,answers)
    
    return questions_data,stud_ans
        
    
def parseIntoExamples(qd,sa):
    
    ret = list()
    
    for index,q_a in qd.items():
        for stud_ans in sa[index]:
            ret.append((q_a[0],q_a[1],stud_ans))
    return ret
                       


    

In [34]:
try:
    path = get_file('shortAnswerGrading-v-2-0.zip',origin='http://web.eecs.umich.edu/~mihalcea/downloads/ShortAnswerGrading_v2.0.zip')
except:
    print "error while downloading file"
    
    
archive = zipfile.ZipFile(path,'r')
    
questions = archive.read('data/sent/questions') 
answers = archive.read('data/sent/answers')
student_answers = archive.read('data/sent/all') 

dic1,dic2 = parseData(questions,answers,student_answers)


data = parseIntoExamples(dic1,dic2)

#print data[0]

vocab = set()
for (_,__,x) in data:
    #for x in el:
    vocab.update(x)
vocab=sorted(list(vocab))

vocab[0:300]



[u'0',
 u'000000',
 u'012',
 u'0x',
 u'1',
 u'10',
 u'100',
 u'115',
 u'123',
 u'12345',
 u'123456789',
 u'12345678910',
 u'124',
 u'154',
 u'18',
 u'1each',
 u'1existing',
 u'1initializing',
 u'1long',
 u'1specification',
 u'1st',
 u'2',
 u'20',
 u'256',
 u'2design',
 u'2n1',
 u'2nd',
 u'2program',
 u'2specifying',
 u'2the',
 u'3',
 u'32',
 u'35',
 u'3n',
 u'3risk',
 u'4',
 u'40',
 u'400',
 u'4verification',
 u'5',
 u'50',
 u'5555',
 u'5657',
 u'5coding',
 u'6',
 u'68',
 u'6testing',
 u'7',
 u'72',
 u'7refining',
 u'8',
 u'80',
 u'88123',
 u'8production',
 u'8th',
 u'9maintenance',
 u'a',
 u'abe',
 u'ability',
 u'able',
 u'about',
 u'above',
 u'abstract',
 u'abstraction',
 u'accepted',
 u'accepts',
 u'accesed',
 u'access',
 u'accessed',
 u'accessible',
 u'accessing',
 u'accessmodifier',
 u'accessspecifications',
 u'accessspecifiers',
 u'accomplish',
 u'accordance',
 u'according',
 u'accordingly',
 u'account',
 u'accurate',
 u'achieved',
 u'acknowledged',
 u'acordingly',
 u'across',
 u

A faire:
- Nettoyer le vocabulaire pour diminuer sa taille, synonymes, lowercase
